In [24]:
import os
import numpy as np
import pandas as pd
import io
import requests


In [4]:
class Action_Space:
    def sample(self):
        return np.random.choice([0, 1])

In [17]:
import ssl
import urllib.request

context = ssl._create_unverified_context()

url = 'https://certificate.tpq.io/rl4finance.csv'
response = urllib.request.urlopen(url, context=context)

In [25]:
class Finance():
    url = 'https://certificate.tpq.io/rl4finance.csv'
    def __init__(self, symbol, feature, n_features=4, min_accuracy=0.085):
        self.action_space = Action_Space()
        self.min_accuracy = min_accuracy
        self.feature = feature
        self.n_features = n_features
        self.symbol = symbol
        self.get_data()
        self._prepare_data()
        
    def get_data(self):
        s = requests.get(url).content
        self.raw = pd.read_csv(io.StringIO(s.decode('utf-8')), index_col=0, parse_dates=True)
        # self.raw = pd.read_csv(self.url, index_col=0, parse_dates=True, verify=False)
        
    def _prepare_data(self):
        self.data = pd.DataFrame(self.raw[self.symbol]).dropna()
        self.data['inc'] = np.log(self.data / self.data.shift(1))
        self.data['act'] = np.where(self.data['inc'] > 0, 1, 0)
        self.data.dropna(inplace=True)
        self.data_ = (self.data - self.data.mean()) / self.data.std()

In [31]:
class Finance(Finance):
    def reset(self):
        self.bar = self.n_features
        self.reward = 0
        state = self.data_[self.feature].iloc[self.bar-self.n_features : self.bar].values
        return state, {}
    
    def step(self, action):
        correct = False
        if action == self.data['act'].iloc[self.bar]:
            correct = True
            
        reward = 1 if correct else 0
        self.reward += reward
        self.bar += 1
        self.accuracy = self.reward / (self.bar - self.n_features)
        if self.bar > len(self.data):
            done = True
        elif self.accuracy < self.min_accuracy and self.bar > 10:
            done = True
        else: done = False
        
        next_state = self.data_[self.feature].iloc[
            self.bar - self.n_features:self.bar].values
        return next_state, reward, done, False, {}

In [32]:
fin = Finance(symbol='EUR=', feature='EUR=')

In [33]:
list(fin.raw.columns)

['AAPL.O',
 'MSFT.O',
 'INTC.O',
 'AMZN.O',
 'GS.N',
 '.SPX',
 '.VIX',
 'SPY',
 'EUR=',
 'XAU=',
 'GDX',
 'GLD']

In [34]:
fin.reset()

(array([2.74844931, 2.64643904, 2.69560062, 2.68085214]), {})

In [35]:
fin.step(fin.action_space.sample())

(array([2.64643904, 2.69560062, 2.68085214, 2.63046153]), 0, False, False, {})